In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web

import pickle
import requests
import bs4 as bs

import datetime as dt
import os

In [2]:
from tqdm import tqdm

import yfinance as yfin
yfin.pdr_override()

In [3]:
def save_sp500_tickers():
    resp = requests.get(
        'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'id': 'constituents'})

    tickers = []
    for row in table.findAll('tr')[1:]:
        tickers.append(row.findAll('td')[0].text.replace('\n', ''))

    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)

    return np.array(tickers)

In [4]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2000, 1, 1)
    end = dt.datetime(2016, 12, 31)

    for ticker in tqdm(tickers):
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.get_data_yahoo(ticker, start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

In [5]:
# get_data_from_yahoo()

In [6]:
def compile_data():
    with open("sp500tickers.pickle", 'rb') as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for ticker in tqdm(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)

        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'],
                axis=1, inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')
    main_df.to_csv('sp500_joined_adjcloses.csv')

In [7]:
compile_data()

100%|███████████| 505/505 [00:24<00:00, 20.22it/s]


In [8]:
df = pd.read_csv('sp500_joined_adjcloses.csv')
df.head().T

,0,1,2,3,4
Date,1999-12-31,2000-01-03,2000-01-04,2000-01-05,2000-01-06
MMM,NaN,26.960117,25.888851,26.638727,28.781269
ABT,NaN,9.418205,9.149119,9.132298,9.451843
ABBV,NaN,NaN,NaN,NaN,NaN
ABMD,NaN,18.25,17.8125,18.0,18.03125
...,...,...,...,...,...
YUM,5.056148,4.884338,4.78616,4.810703,4.769797
ZBRA,26.0,25.027779,24.666668,25.138889,23.777779
ZBH,NaN,NaN,NaN,NaN,NaN
ZION,41.592228,39.000938,37.1124,37.068459,37.59549
